# Data Wrangling

Data wrangling comprises a substantial portion of every data professional's life. Wrangling data encompasses the steps you undertake to organize and clean your underlying data for your analysis. Wrangling includes merging and appending datasets, finding typos, and creating new variables.

## Tidy data

Before we get to the analysis stage of a project, you'll likely first want to arrive at a "cleaned" dataset. Some may call this an "analysis" file, which implies you can start doing analysis without much more fuss. The exact structure of such an analysis file will vary between projects, but there are some fundamental concepts that are common across many situations. Let's start by taking a look at this example from the book, _R for Data Science_ by Hadley Wickham ([Chapter 12](https://r4ds.had.co.nz/tidy-data.html)). Consider the following tables of data.

In [3]:
* Table (1)
frames create infections_tidy
frame infections_tidy {
    input str11 country    year    cases   population
          "Afghanistan"    1999      745     19987071  
          "Afghanistan"    2000     2666     20595360  
          "Brazil"         1999    37737    172006362  
          "Brazil"         2000    80488    174504898  
          "China"          1999   212258   1272915272  
          "China"          2000   213766   1280428583
    end
}

* Table (2)
frames create infections_too_long
frame infections_too_long {
    input str11 country        year  str10 variable     value 
          "Afghanistan"        1999  "cases"              745 
          "Afghanistan"        1999  "population"    19987071 
          "Afghanistan"        2000  "cases"             2666 
          "Afghanistan"        2000  "population"    20595360 
          "Brazil"             1999  "cases"            37737 
          "Brazil"             1999  "population"   172006362 
          "Brazil"             2000  "cases"            80488 
          "Brazil"             2000  "population"   174504898 
          "China"              1999  "cases"           212258 
          "China"              1999  "population"  1272915272 
          "China"              2000  "cases"           213766 
          "China"              2000  "population"  1280428583
    end
}

* Table (3a)
frames create infections_just_cases
frame infections_just_cases {
    input str11 country    v1999    v2000
          "Afghanistan"      745     2666 
          "Brazil"         37737    80488 
          "China"         212258   213766
    end
}

* Table (3b)
frames create infections_just_population
frame infections_just_population {
    input str11 country      v1999        v2000
          "Afghanistan"   19987071     20595360 
          "Brazil"       172006362    174504898 
          "China"       1272915272   1280428583
    end
}




         country       year      cases  populat~n



         country       year    variable      value



         country      v1999      v2000



         country      v1999      v2000


If you wanted to calculate the infection rate (cases per population) and plot it over time by country, which dataset makes that the easiest? Key factors in making a dataset "tidy":

- Each variable must have its own column.
- Each observation must have its own row.
- Each value must have its own cell.

In [4]:
* Table (2)
frames change infections_too_long
reshape wide value, i(country year) j(variable) string
list



(j = cases population)

Data                               Long   ->   Wide
-----------------------------------------------------------------------------
Number of observations               12   ->   6           
Number of variables                   4   ->   4           
j variable (2 values)          variable   ->   (dropped)
xij variables:
                                  value   ->   valuecases valuepopulation
-----------------------------------------------------------------------------


     +------------------------------------------+
     |     country   year   valuec~s   valuep~n |
     |------------------------------------------|
  1. | Afghanistan   1999        745   2.00e+07 |
  2. | Afghanistan   2000       2666   2.06e+07 |
  3. |      Brazil   1999      37737   1.72e+08 |
  4. |      Brazil   2000      80488   1.75e+08 |
  5. |       China   1999     212258   1.27e+09 |
     |------------------------------------------|
  6. |       China   2000     213766   1.28e+09 

In [5]:
* Tables (3a) + (3b)

* Step (1): Reshape to long
* Step (2): Rename variables
frames change infections_just_cases
reshape long v, i(country) j(year)
rename v cases
list
* Save to tempfile to illustrate merge instead of frget/frlink
tempfile infections_just_cases
save `infections_just_cases'

frames change infections_just_population
reshape long v, i(country) j(year)
rename v population
list

* Step (3): Join the data
merge 1:1 country year using `infections_just_cases'
list



(j = 1999 2000)

Data                               Wide   ->   Long
-----------------------------------------------------------------------------
Number of observations                3   ->   6           
Number of variables                   3   ->   3           
j variable (2 values)                     ->   year
xij variables:
                            v1999 v2000   ->   v
-----------------------------------------------------------------------------



     +-----------------------------+
     |     country   year    cases |
     |-----------------------------|
  1. | Afghanistan   1999      745 |
  2. | Afghanistan   2000     2666 |
  3. |      Brazil   1999    37737 |
  4. |      Brazil   2000    80488 |
  5. |       China   1999   212258 |
     |-----------------------------|
  6. |       China   2000   213766 |
     +-----------------------------+


file /var/folders/0w/ccbmzx092b5_9bnw8th_jccw0000gn/T//St15000.000001 saved as
    .dta format


(j = 1999 2000)

Data       

## Tidy values

The previous section discussed the overall structure of the dataset. Now let's dig into common problems you'll run into with actual values (cells) of your data. We can divide these into two categories:

- Strings
- Numeric

There are of course other types of data types (dates, boolean and factors), but let's just focus on these main types for the moment.

### Strings

Strings can be very messy! Let's take the simple example of just asking people to write down what campus they attend.

In [6]:
clear all
input student_id  str14 campus 
               1  "BERKELEY"       // Capitalization
               2  "UCLA "          // Trailing space
               3  " UCLA"          // Leading space            
               4  "U.C.L.A."       // Punctuation
               5  "Berkely "       // Typos
               6  "UC  Berkeley"   // Embedded blanks
               7  "UCB"            // Abbreviations
               8  "Not applicable" // Missing
end




     student~d          campus


In [7]:
* Let's make all campuses uppercase
gen consistent_campus = strupper(campus)
list




     +--------------------------------------------+
     | studen~d           campus   consistent_c~s |
     |--------------------------------------------|
  1. |        1         BERKELEY         BERKELEY |
  2. |        2            UCLA             UCLA  |
  3. |        3             UCLA             UCLA |
  4. |        4         U.C.L.A.         U.C.L.A. |
  5. |        5         Berkely          BERKELY  |
     |--------------------------------------------|
  6. |        6     UC  Berkeley     UC  BERKELEY |
  7. |        7              UCB              UCB |
  8. |        8   Not applicable   NOT APPLICABLE |
     +--------------------------------------------+


In [8]:
* Let's get rid of the leading/trailing spaces
replace consistent_campus = strtrim(consistent_campus)

* And get rid of consecutive spaces
replace consistent_campus = itrim(consistent_campus)

list


(3 real changes made)

(1 real change made)


     +--------------------------------------------+
     | studen~d           campus   consistent_c~s |
     |--------------------------------------------|
  1. |        1         BERKELEY         BERKELEY |
  2. |        2            UCLA              UCLA |
  3. |        3             UCLA             UCLA |
  4. |        4         U.C.L.A.         U.C.L.A. |
  5. |        5         Berkely           BERKELY |
     |--------------------------------------------|
  6. |        6     UC  Berkeley      UC BERKELEY |
  7. |        7              UCB              UCB |
  8. |        8   Not applicable   NOT APPLICABLE |
     +--------------------------------------------+


In [9]:
* Let's get rid of the periods
replace consistent_campus = subinstr(consistent_campus, ".", "", .)

list


(1 real change made)


     +--------------------------------------------+
     | studen~d           campus   consistent_c~s |
     |--------------------------------------------|
  1. |        1         BERKELEY         BERKELEY |
  2. |        2            UCLA              UCLA |
  3. |        3             UCLA             UCLA |
  4. |        4         U.C.L.A.             UCLA |
  5. |        5         Berkely           BERKELY |
     |--------------------------------------------|
  6. |        6     UC  Berkeley      UC BERKELEY |
  7. |        7              UCB              UCB |
  8. |        8   Not applicable   NOT APPLICABLE |
     +--------------------------------------------+


In [10]:
* Let's fix typos
replace consistent_campus = "BERKELEY" if consistent_campus == "BERKELY"

list


(1 real change made)


     +--------------------------------------------+
     | studen~d           campus   consistent_c~s |
     |--------------------------------------------|
  1. |        1         BERKELEY         BERKELEY |
  2. |        2            UCLA              UCLA |
  3. |        3             UCLA             UCLA |
  4. |        4         U.C.L.A.             UCLA |
  5. |        5         Berkely          BERKELEY |
     |--------------------------------------------|
  6. |        6     UC  Berkeley      UC BERKELEY |
  7. |        7              UCB              UCB |
  8. |        8   Not applicable   NOT APPLICABLE |
     +--------------------------------------------+


In [11]:
* Let's standardize values
replace consistent_campus = "UCB" if regexm(consistent_campus, "[UC]*BERKELEY")

list


(3 real changes made)


     +--------------------------------------------+
     | studen~d           campus   consistent_c~s |
     |--------------------------------------------|
  1. |        1         BERKELEY              UCB |
  2. |        2            UCLA              UCLA |
  3. |        3             UCLA             UCLA |
  4. |        4         U.C.L.A.             UCLA |
  5. |        5         Berkely               UCB |
     |--------------------------------------------|
  6. |        6     UC  Berkeley              UCB |
  7. |        7              UCB              UCB |
  8. |        8   Not applicable   NOT APPLICABLE |
     +--------------------------------------------+


In [12]:
* Cleaning up missing values
replace consistent_campus = "" if consistent_campus == "NOT APPLICABLE"

list


(1 real change made)


     +--------------------------------------+
     | studen~d           campus   consis~s |
     |--------------------------------------|
  1. |        1         BERKELEY        UCB |
  2. |        2            UCLA        UCLA |
  3. |        3             UCLA       UCLA |
  4. |        4         U.C.L.A.       UCLA |
  5. |        5         Berkely         UCB |
     |--------------------------------------|
  6. |        6     UC  Berkeley        UCB |
  7. |        7              UCB        UCB |
  8. |        8   Not applicable            |
     +--------------------------------------+


### String or number?

Sometimes you have number stuck in a string (e.g., `"$1,000"`) and sometimes you have a string posing as a number (e.g., ZIP Codes). Let's go through an example of each and how to solve these issues.

In [13]:
clear all
input  zcta  str7 median_earnings 
       1240   "$33,530" 
       1242   "*" 
      89010   "$26,172" 
      89019   "$36,354"
end




          zcta  median_~s


In [14]:
tostring zcta, format("%05.0f") replace

list


zcta was float now str5


     +------------------+
     |  zcta   median~s |
     |------------------|
  1. | 01240    $33,530 |
  2. | 01242          * |
  3. | 89010    $26,172 |
  4. | 89019    $36,354 |
     +------------------+


In [15]:
destring median_earnings, ignore("$*,") replace

list


median_earnings: characters $ * , removed; replaced as long
(1 missing value generated)


     +------------------+
     |  zcta   median~s |
     |------------------|
  1. | 01240      33530 |
  2. | 01242          . |
  3. | 89010      26172 |
  4. | 89019      36354 |
     +------------------+


### Numeric

Numeric variables can be represented as integers or decimal values (float/double). Sometimes integers depict measurements that represent actual integer values (e.g., number of apartments in a building) and sometimes they represent ordinal values (e.g., survey responses of "strongly disagree", "disagree", etc.).

Let's create an example dataset that contains some of the variables.

In [16]:
clear all
input resident_id  unit_id  age  education  income 
                1        1   35          6   57324 
                2        1   27          5   67366 
                3        2   42        -99   47343 
                4        3   56          4  -43123
end




     residen~d    unit_id        age  education     income


One of the first thing to do with numeric values are making sure you're representing missing values correctly. Many data sources will sometimes encode missing values with a placeholder value like -99. Double check your data documentation to make sure you know how missing values are encoded.

In [17]:
replace education = . if education == -99

list


(1 real change made, 1 to missing)


     +----------------------------------------------+
     | reside~d   unit_id   age   educat~n   income |
     |----------------------------------------------|
  1. |        1         1    35          6    57324 |
  2. |        2         1    27          5    67366 |
  3. |        3         2    42          .    47343 |
  4. |        4         3    56          4   -43123 |
     +----------------------------------------------+


In [18]:
replace income = . if income < 0

list


(1 real change made, 1 to missing)


     +----------------------------------------------+
     | reside~d   unit_id   age   educat~n   income |
     |----------------------------------------------|
  1. |        1         1    35          6    57324 |
  2. |        2         1    27          5    67366 |
  3. |        3         2    42          .    47343 |
  4. |        4         3    56          4        . |
     +----------------------------------------------+


**Warning**: Missing values in Stata are treated as very large values when you do comparisons. For example, if you want to see all the observations with incomes greater than $60,000, you'll get the missing values too!

In [19]:
list if income > 60000


     +----------------------------------------------+
     | reside~d   unit_id   age   educat~n   income |
     |----------------------------------------------|
  2. |        2         1    27          5    67366 |
  4. |        4         3    56          4        . |
     +----------------------------------------------+


You can avoid this behavior as follows.

In [20]:
list if income > 60000 & !missing(income)


     +----------------------------------------------+
     | reside~d   unit_id   age   educat~n   income |
     |----------------------------------------------|
  2. |        2         1    27          5    67366 |
     +----------------------------------------------+


Once you're more confident with the values in your numeric columns, you're ready to do the calculations needed for your analysis! Sometimes this could mean doing transformations of variables.

In [22]:
gen ln_income = log(income)

list


variable ln_income already defined


r(110);
r(110);


In [24]:
collapse (mean) income, by(unit_id)

list




     +------------------+
     | unit_id   income |
     |------------------|
  1. |       1    62345 |
  2. |       2    47343 |
  3. |       3        . |
     +------------------+
